<a href="https://colab.research.google.com/github/swago72/cx-knowledge-assistant/blob/main/cx_knowledge_assistant_week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain chromadb sentence-transformers beautifulsoup4 lxml -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently

In [5]:
!pip install langchain-text-splitters -q

In [2]:
from google.colab import files

print("Select all 13 HTML files from your raw folder")
uploaded = files.upload()

print(f"\n✓ {len(uploaded)} files uploaded:")
for fname in uploaded.keys():
    print(f"  - {fname}")

Select all 13 HTML files from your raw folder


Saving Cancel, pause, or change a subscription on Google Play - Computer - Google Play Help.html to Cancel, pause, or change a subscription on Google Play - Computer - Google Play Help.html
Saving Check the status of a refund request for Google Play - Google Play Help.html to Check the status of a refund request for Google Play - Google Play Help.html
Saving Fix payment issues on your account - Computer - Google Play Help.html to Fix payment issues on your account - Computer - Google Play Help.html
Saving Fix problems downloading apps with basic troubleshooting steps - Google Play Help.html to Fix problems downloading apps with basic troubleshooting steps - Google Play Help.html
Saving Fix problems downloading one app - Google Play Help.html to Fix problems downloading one app - Google Play Help.html
Saving Fix problems with subscriptions - Computer - Google Play Help.html to Fix problems with subscriptions - Computer - Google Play Help.html
Saving Fix problems with the Google Play Sto

In [7]:
from bs4 import BeautifulSoup
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb

# ── STEP 1: Load + clean HTML ───────────────────────────────────

def load_html_files(uploaded_files):
    documents = []
    for filename, content in uploaded_files.items():
        raw_html = content.decode("utf-8", errors="ignore")
        soup = BeautifulSoup(raw_html, "lxml")

        # Strip noise
        for tag in soup(["script", "style", "nav", "header",
                          "footer", "aside", "noscript", "iframe"]):
            tag.decompose()

        clean_text = soup.get_text(separator="\n", strip=True)
        lines = [l for l in clean_text.splitlines() if l.strip()]
        clean_text = "\n".join(lines)

        if len(clean_text) < 100:
            print(f"  ⚠ Skipping {filename} — too short")
            continue

        # Create a short clean name for citations
        short_name = filename.replace(" - Google Play Help.html", "")

        documents.append({
            "content":    clean_text,
            "source":     filename,
            "short_name": short_name,
            "chars":      len(clean_text)
        })
        print(f"  ✓ {short_name} ({len(clean_text):,} chars)")

    print(f"\nTotal documents loaded: {len(documents)}")
    return documents


# ── STEP 2: Chunk ───────────────────────────────────────────────

def chunk_documents(documents):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        separators=["\n\n", "\n", ".", " "]
    )

    chunks = []
    for doc in documents:
        texts = splitter.split_text(doc["content"])
        for i, text in enumerate(texts):
            # Create a safe ID — remove spaces and special chars
            safe_id = doc["source"].replace(" ", "_") \
                                   .replace("-", "_") \
                                   .replace(".", "_") \
                                   [:80]
            chunks.append({
                "text":       text,
                "source":     doc["source"],
                "short_name": doc["short_name"],
                "chunk_id":   f"{safe_id}_chunk_{i}",
                "chunk_num":  i,
                "total":      len(texts)
            })

    print(f"Total chunks created: {len(chunks)}")
    return chunks


# ── STEP 3: Embed + store ───────────────────────────────────────

def embed_and_store(chunks, collection_name="cx_knowledge_base"):
    print("Loading HuggingFace model (downloads once ~80MB)...")
    model = SentenceTransformer("all-MiniLM-L6-v2")
    print("✓ Model ready\n")

    client = chromadb.PersistentClient(path="./chroma_db")
    collection = client.get_or_create_collection(
        name=collection_name,
        metadata={"hnsw:space": "cosine"}
    )

    existing_ids = set(collection.get()["ids"])
    new_chunks = [c for c in chunks
                  if c["chunk_id"] not in existing_ids]

    if not new_chunks:
        print("Already embedded. Nothing to add.")
        return collection, model

    print(f"Embedding {len(new_chunks)} chunks...")
    batch_size = 50

    for i in range(0, len(new_chunks), batch_size):
        batch  = new_chunks[i:i + batch_size]
        texts  = [c["text"] for c in batch]
        embeds = model.encode(texts, show_progress_bar=False).tolist()

        collection.add(
            embeddings=embeds,
            documents=[c["text"] for c in batch],
            metadatas=[{
                "source":     c["source"],
                "short_name": c["short_name"],
                "chunk_num":  c["chunk_num"]
            } for c in batch],
            ids=[c["chunk_id"] for c in batch]
        )
        done = min(i + batch_size, len(new_chunks))
        print(f"  Stored {done}/{len(new_chunks)}")

    print(f"\n✓ ChromaDB contains {collection.count()} total chunks")
    return collection, model


# ── RUN ─────────────────────────────────────────────────────────

print("=" * 55)
print("  CX Knowledge Assistant — Ingestion Pipeline")
print("=" * 55 + "\n")

print("STEP 1: Loading HTML files...")
documents = load_html_files(uploaded)

print("\nSTEP 2: Chunking...")
chunks = chunk_documents(documents)

print("\nSTEP 3: Embedding and storing in ChromaDB...")
collection, embed_model = embed_and_store(chunks)

print("\n✅ Done. Knowledge base is ready for Week 2.")

  CX Knowledge Assistant — Ingestion Pipeline

STEP 1: Loading HTML files...
  ✓ Cancel, pause, or change a subscription on Google Play - Computer (8,602 chars)
  ✓ Check the status of a refund request for Google Play (1,463 chars)
  ✓ Fix payment issues on your account - Computer (8,575 chars)
  ✓ Fix problems downloading apps with basic troubleshooting steps (3,429 chars)
  ✓ Fix problems downloading one app (2,256 chars)
  ✓ Fix problems with subscriptions - Computer (5,600 chars)
  ✓ Fix problems with the Google Play Store app (3,718 chars)
  ✓ Learn about Google Play refund policies (2,753 chars)
  ✓ Problems with in-app purchases (5,327 chars)
  ✓ Report charges you don't recognize (3,906 chars)
  ✓ Request a refund on Google Play (2,735 chars)
  ✓ Subscribe to services or content - Computer (11,263 chars)
  ✓ Subscribe with Google (2,164 chars)

Total documents loaded: 13

STEP 2: Chunking...
Total chunks created: 150

STEP 3: Embedding and storing in ChromaDB...
Loading Hugging

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Model ready

Embedding 150 chunks...
  Stored 50/150
  Stored 100/150
  Stored 150/150

✓ ChromaDB contains 150 total chunks

✅ Done. Knowledge base is ready for Week 2.


In [12]:
def test_query(question, n=3):
    embedding = embed_model.encode(question).tolist()
    results = collection.query(
        query_embeddings=[embedding],
        n_results=n,
        include=["documents", "metadatas", "distances"]
    )

    print(f" ? '{question}'\n")
    for i, (doc, meta, dist) in enumerate(zip(
        results["documents"][0],
        results["metadatas"][0],
        results["distances"][0]
    )):
        score = round((1 - dist) * 100, 1)
        print(f"  Result {i+1} — {meta['short_name']} "
              f"({score}% match)")
        print(f"  {doc[:180]}...")
        print()

# Run all 5 — one per issue type
test_query("How do I request a refund for an app?")
test_query("My subscription was charged after I cancelled it")
test_query("App is stuck on pending and won't download")
test_query("There's a charge on my account I don't recognize")
test_query("I was charged for an in-app purchase I didn't make")

 ? 'How do I request a refund for an app?'

  Result 1 — Request a refund on Google Play (77.6% match)
  .
Get support from the developer of the app
You should
contact the app developer
if:
You have a question about an app.
You made an in-app purchase but it wasn’t delivered or isn’t ...

  Result 2 — Problems with in-app purchases (74.8% match)
  Find app developer contact info
.
Request a refund
Depending on the details of your purchase and issue, you may be able to get a refund. Find more information on
returns and refund...

  Result 3 — Learn about Google Play refund policies (69.8% match)
  request a refund
instead.
Most apps on the Play Store are made by third-party developers, not by Google. The developer can help with purchase issues and can process refunds pursuan...

 ? 'My subscription was charged after I cancelled it'

  Result 1 — Subscribe to services or content - Computer (73.6% match)
  . If the payment issue isn't resolved before any grace period and account hold peri